## DC2: Generate Postage Stamps for set of RA, Dec coordinates

This Notebook demonstrates taking a list of RA, Dec positions and generating postage-stamp cutout images from the coadded images.

This is intended to be runnable at NERSC through the jupyter-dev.nersc.gov interface from a local git clone of the repo in your NERSC directory.   You can run it wherever, with appropriate adjustment of the 'repo' location to point to a place where you have a Butler repo will all of the images.

Based in part on https://github.com/lsst-com/notebooks/blob/master/postage_stamp.ipynb

In [ ]:
import numpy as np

from astropy.table import Table

import lsst.daf.persistence as dafPersist
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage

from astropy.visualization import ZScaleInterval

In [ ]:
# Set plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 8)
zscale = ZScaleInterval()

In [ ]:
repo = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/output'
butler = dafPersist.Butler(repo)

In [ ]:
filt = 'r'
coord_file = 'id_ra_dec_mid_mag_%s.txt' % filt
id_ra_dec = Table.read(coord_file, format='ascii')

# Reformat into afw Geometry sphere points for lookup
coord_list = [afwGeom.SpherePoint(r, d, afwGeom.degrees) for r, d in zip(id_ra_dec['RA'], id_ra_dec['DEC'])]

In [ ]:
def cutout_coadd_ra_dec(butler, ra, dec, filt='r', datasetType='deepCoadd', **kwargs):
    """Produce a cutout from coadd from the given butler at the given RA, Dec in decimal degrees
    
    Trivial wrapper around 'cutout_from_coadd_spherepoint'
    """
    radec = afwGeom.SpherePoint(ra, dec, afwGeom.degrees)
    return cutout_from_coadd_spherepoint(butler, radec, filter=filt, datasetType=datasetType)
    
def cutout_from_coadd_spherepoint(butler, radec, filt='r', datasetType='deepCoadd',
                                  skymap=None, cutoutSideLength=50, **kwargs):
    """Produce a cutout from a coadd at the given afw SpherePoint radec
    

    Parameters
    --
    butler - lsst.daf.persistence.Butler of the data repository
    radec - lsst.afw.geom.SpherePoint coordinates of the center of the cutout.
    filter - Filter of the image to load
    datasetType - 'deepCoadd'  Which type of coadd to load.  Doesn't support 'calexp'
    
    skymap - [optional] Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength - [optional] Side of the cutout region in pixels.
    
    Returns
    --
    MaskedImage
    """
    cutoutSideLength = 50  # pixels
    cutoutSize = afwGeom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("deepCoadd_skyMap")
    
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = afwGeom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = afwGeom.BoxI(xy - cutoutSize//2, cutoutSize)

    coaddId = {'tract': tractInfo.getId(), 'patch': "%d,%d" % patchInfo.getIndex(), 'filter': filt}
    
    cutout_image = butler.get(datasetType+'_sub', bbox=bbox, immediate=True, dataId=coaddId)
    
    return cutout_image

In [ ]:
def show_cutout_image(butler, radec, vmin=None, vmax=None):
    cutout_image = cutout_from_coadd_spherepoint(butler, radec, filter='r', datasetType='deepCoadd')

    xy = cutout_image.getWcs().skyToPixel(radec)
    if vmin is None or vmax is None:
        vmin, vmax = zscale.get_limits(cutout_image.image.array)

    plt.imshow(cutout_image.image.array, vmin=vmin, vmax=vmax, cmap='binary')
    plt.colorbar()
    plt.scatter(xy.getX() - cutout_image.getX0(), xy.getY() - cutout_image.getY0(), color='none', edgecolor='red', marker='o', 
        s=200)
    plt.show()

In [ ]:
# Plot just one
show_cutout_image(butler, coord_list[0])

In [ ]:
vmin, vmax = -0.75, +1.5  # Fix the vmin, vmax to make it easier to compare across postage stamps.
for radec in coord_list:
    show_cutout_image(butler, radec, vmin=vmin, vmax=vmax)